# Démonstration : Attaque CPA sur ECB Mode

**Objectifs** :
- Comprendre la sécurité CPA (Chosen Plaintext Attack)
- Démontrer qu'ECB n'est PAS CPA-sécurisé
- Construire un distinguisher efficace
- Comparer avec CBC et CTR

In [ ]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.backends import default_backend
import secrets

## 1. Rappel : Définition CPA-Security

**Jeu IND-CPA (Indistinguishability under Chosen Plaintext Attack)** :

1. **Setup** : Challenger génère clé $k \xleftarrow{\$} \mathcal{K}$
2. **Phase 1** : Adversaire $\mathcal{A}$ interroge oracle $\textsf{Enc}_k(\cdot)$ sur messages $m_1, \ldots, m_q$ de son choix
3. **Challenge** : $\mathcal{A}$ envoie $(m_0^*, m_1^*)$ avec $|m_0^*| = |m_1^*|$
4. Challenger choisit $b \xleftarrow{\$} \{0,1\}$, renvoie $c^* = \textsf{Enc}_k(m_b^*)$
5. **Phase 2** : $\mathcal{A}$ continue à interroger l'oracle (sauf sur $m_0^*, m_1^*$)
6. **Guess** : $\mathcal{A}$ retourne $b' \in \{0,1\}$

**Adversaire gagne si** : $b' = b$

**Avantage** : $\text{Adv}^{\text{CPA}}(\mathcal{A}) = |\Pr[b' = b] - 1/2|$

**Sécurité CPA** : $\text{Adv}^{\text{CPA}}(\mathcal{A}) \leq \epsilon$ négligeable pour tout $\mathcal{A}$ efficace

## 2. Pourquoi ECB échoue

**ECB Mode** : $c_i = E_k(m_i)$ (chaque bloc chiffré indépendamment)

**Faiblesse** : Blocs identiques → ciphertexts identiques !

**Conséquence** : Un adversaire peut **distinguer** facilement

## 3. Attaque CPA sur ECB : Construction du Distinguisher

**Stratégie de l'adversaire** :

1. **Challenge messages** :
   - $m_0 = $ `AAAAAAAAAAAAAAAA AAAAAAAAAAAAAAAA` (deux blocs identiques)
   - $m_1 = $ `AAAAAAAAAAAAAAAA BBBBBBBBBBBBBBBB` (deux blocs différents)

2. **Observation** :
   - Si $c^*$ a deux blocs identiques → guess $b' = 0$
   - Si $c^*$ a deux blocs différents → guess $b' = 1$

3. **Succès** : $\Pr[b' = b] = 1$ → Avantage = 1/2 (maximal !)

In [ ]:
# Oracle de chiffrement ECB
class ECBOracle:
    def __init__(self):
        self.key = secrets.token_bytes(16)  # AES-128
        self.query_count = 0
    
    def encrypt(self, plaintext: bytes) -> bytes:
        """
        Oracle de chiffrement AES-ECB.
        """
        self.query_count += 1
        
        # Padding PKCS7
        padder = padding.PKCS7(128).padder()
        padded = padder.update(plaintext) + padder.finalize()
        
        # ECB encryption
        cipher = Cipher(algorithms.AES(self.key), modes.ECB(), backend=default_backend())
        encryptor = cipher.encryptor()
        ciphertext = encryptor.update(padded) + encryptor.finalize()
        
        return ciphertext

In [ ]:
def cpa_attack_ecb():
    """
    Adversaire CPA contre ECB mode.
    """
    print("=" * 70)
    print("ATTAQUE CPA sur AES-ECB")
    print("=" * 70)
    
    # Setup : Challenger crée l'oracle
    oracle = ECBOracle()
    
    # Phase 1 : Adversaire peut interroger l'oracle (optionnel ici)
    print("\n📋 Phase 1 : Adversaire interroge l'oracle (skip ici)")
    
    # Challenge : Adversaire choisit m0 et m1
    print("\n🎯 Challenge : Adversaire choisit deux messages")
    
    # m0 : Deux blocs IDENTIQUES (16 bytes chacun)
    m0 = b"A" * 16 + b"A" * 16
    
    # m1 : Deux blocs DIFFÉRENTS
    m1 = b"A" * 16 + b"B" * 16
    
    print(f"   m0 = {m0.hex()}")
    print(f"      = {'A'*16} | {'A'*16} (blocs identiques)")
    print(f"\n   m1 = {m1.hex()}")
    print(f"      = {'A'*16} | {'B'*16} (blocs différents)")
    
    # Challenger choisit b aléatoirement
    b = secrets.randbelow(2)
    mb = m0 if b == 0 else m1
    
    print(f"\n🎲 Challenger choisit b = {b} (secret)")
    print(f"   Chiffre m{b}")
    
    # Challenge ciphertext
    c_star = oracle.encrypt(mb)
    
    print(f"\n📦 Challenge ciphertext c*:")
    print(f"   {c_star.hex()}")
    
    # Adversaire analyse c* : compare les deux premiers blocs
    block1 = c_star[:16]
    block2 = c_star[16:32]
    
    print(f"\n🔍 Analyse du ciphertext :")
    print(f"   Bloc 1 : {block1.hex()}")
    print(f"   Bloc 2 : {block2.hex()}")
    
    # Distinguisher
    if block1 == block2:
        b_guess = 0
        print(f"\n   → Blocs IDENTIQUES ✅")
        print(f"   → Adversaire devine : b' = 0 (m0 était chiffré)")
    else:
        b_guess = 1
        print(f"\n   → Blocs DIFFÉRENTS ✅")
        print(f"   → Adversaire devine : b' = 1 (m1 était chiffré)")
    
    # Vérification
    success = (b_guess == b)
    
    print(f"\n" + "=" * 70)
    print(f"RÉSULTAT")
    print("=" * 70)
    print(f"   Bit secret b     : {b}")
    print(f"   Guess b'         : {b_guess}")
    print(f"   Succès           : {success} {'✅' if success else '❌'}")
    print(f"   Requêtes oracle  : {oracle.query_count}")
    
    return success

# Exécution
success = cpa_attack_ecb()

print(f"\n💡 Avec cette attaque, Pr[succès] = 1 (déterministe !)")
print(f"   Avantage = |1 - 1/2| = 1/2 (maximal)")
print(f"\n❌ ECB n'est PAS CPA-sécurisé !")

## 4. Répéter l'expérience

Vérifions que l'attaque réussit **toujours** (probabilité 100%)

In [ ]:
# Répéter l'attaque 100 fois
num_trials = 100
successes = 0

print(f"Exécution de l'attaque {num_trials} fois...\n")

for i in range(num_trials):
    oracle = ECBOracle()
    
    m0 = b"A" * 16 + b"A" * 16
    m1 = b"A" * 16 + b"B" * 16
    
    b = secrets.randbelow(2)
    mb = m0 if b == 0 else m1
    c_star = oracle.encrypt(mb)
    
    block1 = c_star[:16]
    block2 = c_star[16:32]
    
    b_guess = 0 if block1 == block2 else 1
    
    if b_guess == b:
        successes += 1

success_rate = successes / num_trials

print(f"📊 Résultats :")
print(f"   Succès     : {successes}/{num_trials}")
print(f"   Taux       : {success_rate:.2%}")
print(f"   Avantage   : |{success_rate:.2f} - 0.5| = {abs(success_rate - 0.5):.2f}")
print(f"\n✅ Attaque réussit 100% du temps (déterministe) !")

## 5. Comparaison : CBC et CTR sont CPA-sécurisés

**CBC avec IV aléatoire** : Deux chiffrements du même message donnent des ciphertexts différents

**CTR avec nonce unique** : Idem

In [ ]:
def test_cbc_ctr():
    """
    Vérifie que la même attaque NE FONCTIONNE PAS sur CBC et CTR.
    """
    key = secrets.token_bytes(16)
    
    m0 = b"A" * 16 + b"A" * 16
    m1 = b"A" * 16 + b"B" * 16
    
    print("=" * 70)
    print("TEST : CBC et CTR résistent à l'attaque ECB")
    print("=" * 70)
    
    # CBC
    print("\n1️⃣ Mode CBC :")
    
    iv1 = secrets.token_bytes(16)
    iv2 = secrets.token_bytes(16)
    
    # Chiffrer m0 deux fois avec IV différents
    cipher_cbc1 = Cipher(algorithms.AES(key), modes.CBC(iv1), backend=default_backend())
    c1 = cipher_cbc1.encryptor().update(m0) + cipher_cbc1.encryptor().finalize()
    
    cipher_cbc2 = Cipher(algorithms.AES(key), modes.CBC(iv2), backend=default_backend())
    c2 = cipher_cbc2.encryptor().update(m0) + cipher_cbc2.encryptor().finalize()
    
    print(f"   Même message (m0) chiffré 2 fois :")
    print(f"   Ciphertext 1 : {c1[:32].hex()}")
    print(f"   Ciphertext 2 : {c2[:32].hex()}")
    print(f"   Identiques ? {c1 == c2} ❌")
    print(f"   ✅ CBC : ciphertexts différents (grâce aux IV différents)")
    
    # CTR
    print("\n2️⃣ Mode CTR :")
    
    nonce1 = secrets.token_bytes(16)
    nonce2 = secrets.token_bytes(16)
    
    cipher_ctr1 = Cipher(algorithms.AES(key), modes.CTR(nonce1), backend=default_backend())
    c1 = cipher_ctr1.encryptor().update(m0) + cipher_ctr1.encryptor().finalize()
    
    cipher_ctr2 = Cipher(algorithms.AES(key), modes.CTR(nonce2), backend=default_backend())
    c2 = cipher_ctr2.encryptor().update(m0) + cipher_ctr2.encryptor().finalize()
    
    print(f"   Même message (m0) chiffré 2 fois :")
    print(f"   Ciphertext 1 : {c1.hex()}")
    print(f"   Ciphertext 2 : {c2.hex()}")
    print(f"   Identiques ? {c1 == c2} ❌")
    print(f"   ✅ CTR : ciphertexts différents (grâce aux nonces différents)")
    
    print(f"\n💡 CBC et CTR sont **randomisés** → CPA-sécurisés")
    print(f"   L'attaque sur ECB NE FONCTIONNE PAS ici !")

test_cbc_ctr()

## 6. Autre attaque CPA : Détection de patterns

**Scénario** : Un attaquant veut savoir si un utilisateur a recherché un mot spécifique

**Stratégie** : Interroger l'oracle avec le mot suspect et comparer

In [ ]:
def pattern_detection_attack():
    """
    Attaque par détection de patterns sur ECB.
    """
    print("=" * 70)
    print("ATTAQUE : Détection de Patterns (ECB)")
    print("=" * 70)
    
    oracle = ECBOracle()
    
    # Message secret de la victime (inconnu de l'attaquant)
    secret_search = b"classified      "  # 16 bytes (padding inclus)
    
    # Victime chiffre sa recherche
    c_victim = oracle.encrypt(secret_search)
    
    print(f"\n🎯 Victime chiffre sa recherche (secret) :")
    print(f"   Ciphertext : {c_victim[:16].hex()}")
    
    # Attaquant interroge l'oracle avec des mots suspects
    suspects = [b"classified      ", b"confidential    ", b"secret          "]
    
    print(f"\n🔍 Attaquant teste des mots suspects :")
    
    for word in suspects:
        c_test = oracle.encrypt(word)
        match = (c_test[:16] == c_victim[:16])
        
        status = "✅ MATCH" if match else "❌ Différent"
        print(f"   {word.decode().strip():15} → {status}")
        
        if match:
            print(f"\n🔓 Attaquant a trouvé : La victime a recherché '{word.decode().strip()}' !")
    
    print(f"\n⚠️  ECB permet de détecter des patterns → perte de confidentialité")

pattern_detection_attack()

## Conclusion

**Points clés** :
- **CPA (Chosen Plaintext Attack)** : Modèle réaliste où l'adversaire peut chiffrer des messages
- **ECB échoue spectaculairement** : Avantage = 1/2 (maximal), attaque déterministe
- **Blocs identiques → Ciphertexts identiques** : Révèle la structure du message
- **CBC et CTR résistent** : Randomisation via IV/nonce

**Leçons** :
- ❌ JAMAIS utiliser ECB en pratique
- ✅ Utiliser CBC (avec IV aléatoire) ou CTR (avec nonce unique)
- ✅ En production : AEAD (AES-GCM, ChaCha20-Poly1305)

**Applications réelles** :
- Attaque célèbre : Chiffrement ECB d'images (contours visibles)
- Détection de cookies/tokens réutilisés
- Analyse de trafic chiffré (patterns)